In [ ]:
from .database import get_connection

def log_prediction(prompt: str, result: dict):
    conn = get_connection()
    cursor = conn.cursor()

    query = """
        INSERT INTO jailbreak_logs (prompt, label, benign_score, jailbreak_score)
        VALUES (%s, %s, %s, %s)
    """
    cursor.execute(query, (
        prompt,
        result["label"],
        result["benign_score"],
        result["jailbreak_score"],
    ))

    conn.commit()
    cursor.close()
    conn.close()


def get_recent_logs(limit=20):
    conn = get_connection()
    cursor = conn.cursor(dictionary=True)

    query = f"""
        SELECT prompt, label, benign_score, jailbreak_score,
               DATE_FORMAT(timestamp, '%Y-%m-%d %H:%i') AS timestamp
        FROM jailbreak_logs
        ORDER BY id DESC
        LIMIT {limit}
    """

    cursor.execute(query)
    rows = cursor.fetchall()

    cursor.close()
    conn.close()
    return rows


def get_summary_stats():
    conn = get_connection()
    cursor = conn.cursor(dictionary=True)

    # Total counts
    cursor.execute("""
        SELECT
            COUNT(*) AS total,
            SUM(label='benign') AS benign,
            SUM(label='jailbreak') AS jailbreak
        FROM jailbreak_logs
    """)
    summary = cursor.fetchone()

    # Timeline per day
    cursor.execute("""
        SELECT
            DATE(timestamp) AS date,
            SUM(label='benign') AS benign,
            SUM(label='jailbreak') AS jailbreak
        FROM jailbreak_logs
        GROUP BY DATE(timestamp)
        ORDER BY DATE(timestamp)
    """)
    timeline = cursor.fetchall()

    cursor.close()
    conn.close()

    summary["timeline"] = timeline
    return summary
